In [1]:
import pandas as pd

transactions = pd.read_csv("../data/processed/clean_transactions.csv")
rfm = pd.read_csv("../data/processed/rfm_clustered.csv")

transactions.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


In [2]:
transactions = transactions.merge(
    rfm[['CustomerID', 'Cluster']],
    left_on='Customer ID',
    right_on='CustomerID',
    how='inner'
)

transactions.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,CustomerID,Cluster
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4,13085,1
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,13085,1
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,13085,1
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8,13085,1
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,13085,1


In [3]:
vip_data = transactions[transactions['Cluster'] == 3]
print(vip_data.shape)


(45247, 11)


In [4]:
basket = (
    vip_data
    .groupby(['Invoice', 'Description'])['Quantity']
    .sum()
    .unstack()
    .fillna(0)
)

basket = basket.applymap(lambda x: 1 if x > 0 else 0)

basket.head()


/var/folders/8m/ywn8c31s2gq_1g_fc4hjrgf00000gn/T/ipykernel_76150/1260629923.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


Description,DOORMAT UNION JACK GUNS AND ROSES,3 STRIPEY MICE FELTCRAFT,4 PURPLE FLOCK DINNER CANDLES,BLACK PIRATE TREASURE CHEST,CHERRY BLOSSOM DECORATIVE FLASK,FLAMINGO LIGHTS,HOME SWEET HOME BLACKBOARD,IVORY PAPER CUP CAKE CASES,PAINT YOUR OWN CANVAS SET,PEACE WOODEN BLOCK LETTERS,...,ZINC FINISH 15CM PLANTER POTS,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE DOUBLE PLANTER,ZINC HEART LATTICE PLANTER BOWL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART LATTICE TRAY OVAL,ZINC METAL HEART DECORATION,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK
Invoice,,,,,,,,,,,,,,,,,,,,,
489436,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
489465,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489514,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489523,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489539,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
basket.shape

(2573, 3305)

In [6]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(
    basket,
    min_support=0.03,
    use_colnames=True
)

print(frequent_itemsets.shape)
frequent_itemsets.sort_values(by='support', ascending=False).head(10)


(90, 2)


/Users/talindaga/Desktop/ML/venv/lib/python3.14/site-packages/mlxtend/frequent_patterns/fpcommon.py:175: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
62,0.158958,(WHITE HANGING HEART T-LIGHT HOLDER)
47,0.081228,(REGENCY CAKESTAND 3 TIER)
66,0.079674,(WOODEN FRAME ANTIQUE WHITE )
65,0.075787,(WOOD S/3 CABINET ANT WHITE FINISH)
56,0.071512,(STRAWBERRY CERAMIC TRINKET BOX)
67,0.071123,(WOODEN PICTURE FRAME WHITE FINISH)
31,0.067237,(LUNCH BAG RED SPOTTY)
2,0.063739,(ASSORTED COLOUR BIRD ORNAMENT)
19,0.062962,(JUMBO BAG RED RETROSPOT)
46,0.061407,(RED WOOLLY HOTTIE WHITE HEART.)


In [7]:
rules = association_rules(
    frequent_itemsets,
    metric="lift",
    min_threshold=1.2
)

rules = rules.sort_values(by='confidence', ascending=False)

rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)


,antecedents,consequents,support,confidence,lift
56,"(WOODEN PICTURE FRAME WHITE FINISH, WOOD 2 DRA...",(WOODEN FRAME ANTIQUE WHITE ),0.046638,0.991736,12.447490
69,"(WOODEN PICTURE FRAME WHITE FINISH, WOOD 2 DRA...",(WOODEN FRAME ANTIQUE WHITE ),0.043529,0.991150,12.440147
70,"(WOODEN PICTURE FRAME WHITE FINISH, WOODEN FRA...",(WOOD 2 DRAWER CABINET WHITE FINISH),0.043529,0.982456,16.414673
71,"(WOOD 2 DRAWER CABINET WHITE FINISH, WOODEN FR...",(WOODEN PICTURE FRAME WHITE FINISH),0.043529,0.965517,13.575278
58,"(WOOD 2 DRAWER CABINET WHITE FINISH, WOODEN FR...",(WOODEN PICTURE FRAME WHITE FINISH),0.046638,0.960000,13.497705
45,"(WOOD 2 DRAWER CABINET WHITE FINISH, WOOD S/3 ...",(WOODEN FRAME ANTIQUE WHITE ),0.045084,0.935484,11.741463
50,"(WOODEN PICTURE FRAME WHITE FINISH, WOOD 2 DRA...",(WOOD S/3 CABINET ANT WHITE FINISH),0.043918,0.933884,12.322484
68,"(WOODEN PICTURE FRAME WHITE FINISH, WOOD 2 DRA...",(WOOD S/3 CABINET ANT WHITE FINISH),0.043529,0.933333,12.315214
44,"(WOOD 2 DRAWER CABINET WHITE FINISH, WOODEN FR...",(WOOD S/3 CABINET ANT WHITE FINISH),0.045084,0.928000,12.244841
63,"(WOODEN PICTURE FRAME WHITE FINISH, WOOD S/3 C...",(WOODEN FRAME ANTIQUE WHITE ),0.044306,0.926829,11.632838


In [8]:
strong_rules = rules[
    (rules['confidence'] >= 0.6) &
    (rules['lift'] >= 1.3)
]

strong_rules[['antecedents', 'consequents', 'confidence', 'lift']].head(10)


,antecedents,consequents,confidence,lift
56,"(WOODEN PICTURE FRAME WHITE FINISH, WOOD 2 DRA...",(WOODEN FRAME ANTIQUE WHITE ),0.991736,12.447490
69,"(WOODEN PICTURE FRAME WHITE FINISH, WOOD 2 DRA...",(WOODEN FRAME ANTIQUE WHITE ),0.991150,12.440147
70,"(WOODEN PICTURE FRAME WHITE FINISH, WOODEN FRA...",(WOOD 2 DRAWER CABINET WHITE FINISH),0.982456,16.414673
71,"(WOOD 2 DRAWER CABINET WHITE FINISH, WOODEN FR...",(WOODEN PICTURE FRAME WHITE FINISH),0.965517,13.575278
58,"(WOOD 2 DRAWER CABINET WHITE FINISH, WOODEN FR...",(WOODEN PICTURE FRAME WHITE FINISH),0.960000,13.497705
45,"(WOOD 2 DRAWER CABINET WHITE FINISH, WOOD S/3 ...",(WOODEN FRAME ANTIQUE WHITE ),0.935484,11.741463
50,"(WOODEN PICTURE FRAME WHITE FINISH, WOOD 2 DRA...",(WOOD S/3 CABINET ANT WHITE FINISH),0.933884,12.322484
68,"(WOODEN PICTURE FRAME WHITE FINISH, WOOD 2 DRA...",(WOOD S/3 CABINET ANT WHITE FINISH),0.933333,12.315214
44,"(WOOD 2 DRAWER CABINET WHITE FINISH, WOODEN FR...",(WOOD S/3 CABINET ANT WHITE FINISH),0.928000,12.244841
63,"(WOODEN PICTURE FRAME WHITE FINISH, WOOD S/3 C...",(WOODEN FRAME ANTIQUE WHITE ),0.926829,11.632838


Association rule mining on VIP customers revealed extremely strong co-purchase behavior among white and wooden home décor products. Several product combinations exhibited confidence values above 95% and lift values exceeding 10, indicating intentional and style-driven purchasing. These findings support the creation of curated product bundles and recommendation strategies targeted at high-value customers.